In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

In [2]:
dev = qml.device('strawberryfields.fock', wires=4, cutoff_dim=4)

In [3]:
def layer(theta, phi, wires):
    M = len(wires)
    phi_nonlinear = np.pi / 2

    qml.Interferometer(
        theta, phi, np.zeros(M), wires=wires, mesh="triangular",
    )

    for i in wires:
        qml.Kerr(phi_nonlinear, wires=i)

In [4]:
@qml.qnode(dev)
def quantum_neural_net(var, x):
    wires = list(range(len(x)))

    for i in wires:
        qml.FockState(x[i], wires=i)

    # "layer" subcircuits
    for i, v in enumerate(var):
        layer(v[: len(v) // 2], v[len(v) // 2 :], wires)

    return [qml.expval(qml.NumberOperator(w)) for w in wires]

In [5]:
def square_loss(labels, predictions):
    term = 0
    for l, p in zip(labels, predictions):
        lnorm = l / np.linalg.norm(l)
        pnorm = p / np.linalg.norm(p)

        term = term + np.abs(np.dot(lnorm, pnorm.T)) ** 2

    return 1 - term / len(labels)

In [6]:
def cost(var, data_input, labels):
    predictions = np.array([quantum_neural_net(var, x) for x in data_input])
    sl = square_loss(labels, predictions)

    return sl

In [7]:
X = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 1, 0],
              [0, 1, 0, 1]], requires_grad=False)

Y = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 1, 1, 0]], requires_grad=False)

In [8]:
num_layers = 2
M = len(X[0])
num_variables_per_layer = M * (M - 1)

rng = np.random.default_rng(seed=1234)
var_init = (4 * rng.random(size=(num_layers, num_variables_per_layer), requires_grad=True) - 2) * np.pi
print(var_init)

[[ 5.99038594 -1.50550479  5.31866903 -2.99466132 -2.27329341 -4.79920711
  -3.24506046 -2.2803699   5.83179179 -2.97006415 -0.74133893  1.38067731]
 [ 4.56939998  4.5711137   2.1976234   2.00904031  2.96261861 -3.48398028
  -4.12093786  4.65477183 -5.52746064  2.30830291  2.15184041  1.3950931 ]]


In [ ]:
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)

var = var_init
for it in range(200):
    var = opt.step(lambda v: cost(v, X, Y), var)

    if (it+1) % 20 == 0:
        print(f"Iter: {it+1:5d} | Cost: {cost(var, X, Y):0.7f} ")

In [ ]:
print(f"The optimized parameters (layers, parameters):\n {var}\n")

Y_pred = np.array([quantum_neural_net(var, x) for x in X])
for i, x in enumerate(X):
    print(f"{x} --> {Y_pred[i].round(2)}, should be {Y[i]}")

In [ ]:
print(qml.draw(quantum_neural_net)(var_init, X[0]))